# Probando LlamaIndex para sitios web

In [36]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [37]:
## Configuración para debugging

from llama_index.core.callbacks import (
    CallbackManager,
    LlamaDebugHandler,
    CBEventType,
)

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

In [38]:
from llama_index.core import Settings

Settings.callback_manager = callback_manager

In [39]:
from llama_index.readers.web import SimpleWebPageReader

documents = SimpleWebPageReader(html_to_text=True).load_data(
    ["https://hacemosescuela.cba.gov.ar/materiales-educativos/"]
)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): hacemosescuela.cba.gov.ar:443
Starting new HTTPS connection (1): hacemosescuela.cba.gov.ar:443
Starting new HTTPS connection (1): hacemosescuela.cba.gov.ar:443
DEBUG:urllib3.connectionpool:https://hacemosescuela.cba.gov.ar:443 "GET /materiales-educativos/ HTTP/1.1" 200 None
https://hacemosescuela.cba.gov.ar:443 "GET /materiales-educativos/ HTTP/1.1" 200 None
https://hacemosescuela.cba.gov.ar:443 "GET /materiales-educativos/ HTTP/1.1" 200 None


In [40]:
len(documents)

1

In [41]:
documents[0]

Document(id_='https://hacemosescuela.cba.gov.ar/materiales-educativos/', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='[ ![Logo](https://hacemosescuela.cba.gov.ar/wp-\ncontent/themes/hacemosescuela_2-8/img/logo_202403.png?v=20240411-1)\n](https://hacemosescuela.cba.gov.ar) __\n\n  * [Inicio](https://hacemosescuela.cba.gov.ar)\n  * [Acerca de](https://hacemosescuela.cba.gov.ar/acerca-de)\n  * Materiales educativos\n    * [Secuencias didácticas](https://hacemosescuela.cba.gov.ar/materiales-educativos)\n    * [Objetos Hipermediales Interactivos](https://hacemosescuela.cba.gov.ar/ohi)\n    * [Colecciones didácticas](https://hacemosescuela.cba.gov.ar/colecciones-didacticas)\n    * [Recomendaciones](https://hacemosescuela.cba.gov.ar/recomendaciones)\n  * Programas\n    * [Entre Lenguas](https://hacemosescuela.cba.gov.ar/entre-lenguas)\n    * [Cultura Digital](https://hacemosescuela.cba.gov.ar/cultura-digital)\n    * [Otro

In [42]:
from llama_index.core.node_parser import MarkdownNodeParser

nodes = MarkdownNodeParser(include_metadata=True, include_prev_next_rel=True).get_nodes_from_documents(documents)


DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: [ ![Logo](https://hacemosescuela.cba.gov.ar/wp-...
> Adding chunk: [ ![Logo](https://hacemosescuela.cba.gov.ar/wp-...
> Adding chunk: [ ![Logo](https://hacemosescuela.cba.gov.ar/wp-...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Materiales educativos

En este espacio encontra...
> Adding chunk: Materiales educativos

En este espacio encontra...
> Adding chunk: Materiales educativos

En este espacio encontra...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Educación Inicial

[](https://hacemosescuela.cb...
> Adding chunk: Educación Inicial

[](https://hacemosescuela.cb...
> Adding chunk: Educación Inicial

[](https://hacemosescuela.cb...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Educación Primaria

[](https://hacemosescuela.c...
> Adding chunk: Educación Primaria

[](https://hacemosescuela.c...
> Adding chunk: Educación Primaria

[](https://hacemosescuela.c...
DEBUG:llama_

In [43]:
len(nodes)

7

In [44]:
from llama_index.core.node_parser import (
    HierarchicalNodeParser,
    SentenceSplitter,
)
node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=[2000, 1000, 500])
nodes = node_parser.get_nodes_from_documents(documents)
len(nodes)

DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: [ ![Logo](https://hacemosescuela.cba.gov.ar/wp-...
> Adding chunk: [ ![Logo](https://hacemosescuela.cba.gov.ar/wp-...
> Adding chunk: [ ![Logo](https://hacemosescuela.cba.gov.ar/wp-...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: es una secuencia didáctica de Tu
Escuela en Cas...
> Adding chunk: es una secuencia didáctica de Tu
Escuela en Cas...
> Adding chunk: es una secuencia didáctica de Tu
Escuela en Cas...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Patria, revolución y guerra es una secuencia di...
> Adding chunk: Patria, revolución y guerra es una secuencia di...
> Adding chunk: Patria, revolución y guerra es una secuencia di...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: [ ![Logo](https://hacemosescuela.cba.gov.ar/wp-...
> Adding chunk: [ ![Logo](https://hacemosescuela.cba.gov.ar/wp-...
> Adding chunk: [ ![Logo](https://hacemosescuela.cba.gov.ar/wp-...
DEBUG:llama_

28

In [45]:
from llama_index.core.node_parser import get_leaf_nodes, get_root_nodes

In [46]:
leaf_nodes = get_leaf_nodes(nodes)
len(leaf_nodes)

17

In [47]:
root_nodes = get_root_nodes(nodes)
len(root_nodes)

3

In [48]:
import os
from dotenv import load_dotenv

load_dotenv('.env', override=True)

GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")

In [49]:
# define storage context
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import StorageContext


docstore = SimpleDocumentStore()

# insert nodes into docstore
docstore.add_documents(nodes)

# define storage context (will include vector store by default too)
storage_context = StorageContext.from_defaults(docstore=docstore, )


In [50]:
# imports
from llama_index.embeddings.gemini import GeminiEmbedding

model_name = "models/embedding-001"

embed_model = GeminiEmbedding(
    model_name=model_name, api_key=GOOGLE_API_KEY, title="this is a document"
)

In [51]:
## Load index into vector index
from llama_index.core import VectorStoreIndex

base_index = VectorStoreIndex(
    leaf_nodes,
    storage_context=storage_context,
    embed_model=embed_model
)

**********
Trace: index_construction
    |_CBEventType.EMBEDDING -> 4.075085 seconds
    |_CBEventType.EMBEDDING -> 2.27061 seconds
**********


In [52]:
from llama_index.core.retrievers import AutoMergingRetriever

base_retriever = base_index.as_retriever(similarity_top_k=6,)
retriever = AutoMergingRetriever(base_retriever, storage_context, verbose=True)

In [53]:
# query_str = "What were some lessons learned from red-teaming?"
# query_str = "Can you tell me about the key concepts for safety finetuning"
query_str = (
    "educación primaria"
)

nodes = retriever.retrieve(query_str)

DEBUG:llama_index.core.indices.utils:> Top 6 nodes:
> [Node 7bce84fc-dbd0-4924-aa0b-55d6cb726e50] [Similarity score:             0.789408] es una secuencia didáctica de Tu
Escuela en Casa destinada a las salas de 4...

[ __ Ver más mate...
> [Node 99094bec-32ae-4cc1-8e6c-2f306e917648] [Similarity score:             0.786651] Fueron
diseñados para diferentes escenarios escolares: presencial, no presencial o
combinado. Tie...
> [Node 14938e36-0656-484e-9cae-78e5eb78cbfb] [Similarity score:             0.777371] [ ![Logo](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/logo_202403...
> [Node fb4c3f5c-a56b-494b-8f73-a9e2b05ede5c] [Similarity score:             0.763972] Ofrece un abordaje acerca de un...

[](https://hacemosescuela.cba.gov.ar/te-invito-a-mi-jardin-co...
> [Node af14ee82-9691-4209-94a0-beca7774eba7] [Similarity score:             0.762146] Patria, revolución y
guerra](https://hacemosescuela.cba.gov.ar/efemerides-25-de-mayo-patria-
revo...
> [Node 

In [54]:
base_nodes = base_retriever.retrieve(query_str)

DEBUG:llama_index.core.indices.utils:> Top 6 nodes:
> [Node 7bce84fc-dbd0-4924-aa0b-55d6cb726e50] [Similarity score:             0.789408] es una secuencia didáctica de Tu
Escuela en Casa destinada a las salas de 4...

[ __ Ver más mate...
> [Node 99094bec-32ae-4cc1-8e6c-2f306e917648] [Similarity score:             0.786651] Fueron
diseñados para diferentes escenarios escolares: presencial, no presencial o
combinado. Tie...
> [Node 14938e36-0656-484e-9cae-78e5eb78cbfb] [Similarity score:             0.777371] [ ![Logo](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/logo_202403...
> [Node fb4c3f5c-a56b-494b-8f73-a9e2b05ede5c] [Similarity score:             0.763972] Ofrece un abordaje acerca de un...

[](https://hacemosescuela.cba.gov.ar/te-invito-a-mi-jardin-co...
> [Node af14ee82-9691-4209-94a0-beca7774eba7] [Similarity score:             0.762146] Patria, revolución y
guerra](https://hacemosescuela.cba.gov.ar/efemerides-25-de-mayo-patria-
revo...
> [Node 



---



In [55]:
from llama_index.core.response.notebook_utils import display_source_node

for node in nodes:
    display_source_node(node, source_length=10000)

**Node ID:** 7bce84fc-dbd0-4924-aa0b-55d6cb726e50<br>**Similarity:** 0.7894079288464911<br>**Text:** es una secuencia didáctica de Tu
Escuela en Casa destinada a las salas de 4...

[ __ Ver más materiales del nivel Inicial
](http://hacemosescuela.cba.gov.ar/buscador/?nivel=158&key=)

## Educación Primaria

[](https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-las-y-
los-trabajadores-derechos-reivindicaciones-y-luchas/)

[Efemérides: 1.º de Mayo El trabajo, las y los trabajadores: derechos,
reivindicaciones y
luchas](https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-
las-y-los-trabajadores-derechos-reivindicaciones-y-luchas/)

Efemérides: 1.º de Mayo. El trabajo, las y los trabajadores: derechos,
reivindicaciones y luchas es una compilación de registros...

[](https://hacemosescuela.cba.gov.ar/efemerides-25-de-mayo-aquel-primer-
festejo/)

[Efemérides: 25 de mayo.<br>

**Node ID:** f02edf5d-0683-4c15-a4ea-9dd8a1dd98c8<br>**Similarity:** 0.7820111591650015<br>**Text:** [ ![Logo](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/logo_202403.png?v=20240411-1)
](https://hacemosescuela.cba.gov.ar) __

  * [Inicio](https://hacemosescuela.cba.gov.ar)
  * [Acerca de](https://hacemosescuela.cba.gov.ar/acerca-de)
  * Materiales educativos
    * [Secuencias didácticas](https://hacemosescuela.cba.gov.ar/materiales-educativos)
    * [Objetos Hipermediales Interactivos](https://hacemosescuela.cba.gov.ar/ohi)
    * [Colecciones didácticas](https://hacemosescuela.cba.gov.ar/colecciones-didacticas)
    * [Recomendaciones](https://hacemosescuela.cba.gov.ar/recomendaciones)
  * Programas
    * [Entre Lenguas](https://hacemosescuela.cba.gov.ar/entre-lenguas)
    * [Cultura Digital](https://hacemosescuela.cba.gov.ar/cultura-digital)
    * [Otros programas](https://hacemosescuela.cba.gov.ar/programas)
  * [Comunidad](https://hacemosescuela.cba.gov.ar/comunidad-de-practicas)
  * [Conversatorios](https://hacemosescuela.cba.gov.ar/conversatorios)
  * Modalidades
    * [Jóvenes y Adultos](https://hacemosescuela.cba.gov.ar/jovenes-y-adultos)
    * [Educación Especial](https://hacemosescuela.cba.gov.ar/educacion-especial)
    * [Domiciliaria y Hospitalaria](https://hacemosescuela.cba.gov.ar/educacion-domiciliaria-y-hospitalaria)

![](https://hacemosescuela.cba.gov.ar/wp-content/uploads/2022/11/Banner-
materiales-educativos-1024x267.jpg)

##

## Materiales educativos

En este espacio encontrarán materiales ordenados por nivel educativo. Fueron
diseñados para diferentes escenarios escolares: presencial, no presencial o
combinado. Tienen el propósito de ofrecer desarrollos didácticos para
acompañar la escolaridad y el diseño de propuestas de enseñanza por parte de
los docentes de todos los niveles.

Estos materiales se inscriben en los contenidos y aprendizajes priorizados por
los programas del Ministerio de Educación.

## Educación Inicial

[](https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-las-y-
los-trabajadores-derechos-reivindicaciones-y-luchas/)

[Efemérides: 1.º de Mayo El trabajo, las y los trabajadores: derechos,
reivindicaciones y
luchas](https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-
las-y-los-trabajadores-derechos-reivindicaciones-y-luchas/)

Efemérides: 1.º de Mayo.<br>

**Node ID:** fb4c3f5c-a56b-494b-8f73-a9e2b05ede5c<br>**Similarity:** 0.7639715754382289<br>**Text:** Ofrece un abordaje acerca de un...

[](https://hacemosescuela.cba.gov.ar/te-invito-a-mi-jardin-conocer-y-
describir-nuestro-espacio-de-trabajo/)

[Te invito a mi jardín. Conocer y describir nuestro espacio de
trabajo](https://hacemosescuela.cba.gov.ar/te-invito-a-mi-jardin-conocer-y-
describir-nuestro-espacio-de-trabajo/)

“Te invito a mi jardín.<br>

**Node ID:** d181bf1a-263a-4d1a-a247-211f4013035f<br>**Similarity:** 0.7621458538579383<br>**Text:** Patria, revolución y
guerra](https://hacemosescuela.cba.gov.ar/efemerides-25-de-mayo-patria-
revolucion-y-guerra/)

Efemérides: 25 de Mayo.<br>

**Node ID:** 05612c0d-7bd1-406e-9fea-ca818c3f0355<br>**Similarity:** 0.7618394834503764<br>**Text:** Paseos asombrosos, fotomontajes
lúdicos](https://hacemosescuela.cba.gov.ar/fotografia-en-el-aula-paseos-
asombrosos-fotomontajes-ludicos/)

“Fotografía en el aula. Paseos asombrosos, fotomontajes lúdicos” es una
secuencia didáctica de Hacemos Escuela destinada...

[](https://hacemosescuela.cba.gov.ar/de-la-colmena-a-tu-mesa/)

[De la colmena a tu mesa](https://hacemosescuela.cba.gov.ar/de-la-colmena-a-
tu-mesa/)

"De la colmena a tu mesa" es una secuencia didáctica de Tu Escuela en Casa
destinada a 1.º grado de nivel Primario....

[](https://hacemosescuela.cba.gov.ar/programamos-con-alternativa-condicional/)

[Programamos con alternativa
condicional](https://hacemosescuela.cba.gov.ar/programamos-con-alternativa-
condicional/)

“Programamos con alternativa condicional” es una clase de Hacemos Escuela
destinada al Primer Ciclo del nivel Primario. Propone...

[ __ Ver más materiales del nivel Primario
](http://hacemosescuela.cba.gov.ar/buscador/?nivel=161&key=)

## Educación Secundaria

[](https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-las-y-
los-trabajadores-derechos-reivindicaciones-y-luchas/)

[Efemérides: 1.º de Mayo El trabajo, las y los trabajadores: derechos,
reivindicaciones y
luchas](https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-
las-y-los-trabajadores-derechos-reivindicaciones-y-luchas/)

Efemérides: 1.º de Mayo.<br>

In [56]:
for node in base_nodes:
    display_source_node(node, source_length=10000)

**Node ID:** 7bce84fc-dbd0-4924-aa0b-55d6cb726e50<br>**Similarity:** 0.7894079288464911<br>**Text:** es una secuencia didáctica de Tu
Escuela en Casa destinada a las salas de 4...

[ __ Ver más materiales del nivel Inicial
](http://hacemosescuela.cba.gov.ar/buscador/?nivel=158&key=)

## Educación Primaria

[](https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-las-y-
los-trabajadores-derechos-reivindicaciones-y-luchas/)

[Efemérides: 1.º de Mayo El trabajo, las y los trabajadores: derechos,
reivindicaciones y
luchas](https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-
las-y-los-trabajadores-derechos-reivindicaciones-y-luchas/)

Efemérides: 1.º de Mayo. El trabajo, las y los trabajadores: derechos,
reivindicaciones y luchas es una compilación de registros...

[](https://hacemosescuela.cba.gov.ar/efemerides-25-de-mayo-aquel-primer-
festejo/)

[Efemérides: 25 de mayo.<br>

**Node ID:** 99094bec-32ae-4cc1-8e6c-2f306e917648<br>**Similarity:** 0.7866513892755583<br>**Text:** Fueron
diseñados para diferentes escenarios escolares: presencial, no presencial o
combinado. Tienen el propósito de ofrecer desarrollos didácticos para
acompañar la escolaridad y el diseño de propuestas de enseñanza por parte de
los docentes de todos los niveles.

Estos materiales se inscriben en los contenidos y aprendizajes priorizados por
los programas del Ministerio de Educación.

## Educación Inicial

[](https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-las-y-
los-trabajadores-derechos-reivindicaciones-y-luchas/)

[Efemérides: 1.º de Mayo El trabajo, las y los trabajadores: derechos,
reivindicaciones y
luchas](https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-
las-y-los-trabajadores-derechos-reivindicaciones-y-luchas/)

Efemérides: 1.º de Mayo.<br>

**Node ID:** 14938e36-0656-484e-9cae-78e5eb78cbfb<br>**Similarity:** 0.7773709290544447<br>**Text:** [ ![Logo](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/logo_202403.png?v=20240411-1)
](https://hacemosescuela.cba.gov.ar) __

  * [Inicio](https://hacemosescuela.cba.gov.ar)
  * [Acerca de](https://hacemosescuela.cba.gov.ar/acerca-de)
  * Materiales educativos
    * [Secuencias didácticas](https://hacemosescuela.cba.gov.ar/materiales-educativos)
    * [Objetos Hipermediales Interactivos](https://hacemosescuela.cba.gov.ar/ohi)
    * [Colecciones didácticas](https://hacemosescuela.cba.gov.ar/colecciones-didacticas)
    * [Recomendaciones](https://hacemosescuela.cba.gov.ar/recomendaciones)
  * Programas
    * [Entre Lenguas](https://hacemosescuela.cba.gov.ar/entre-lenguas)
    * [Cultura Digital](https://hacemosescuela.cba.gov.ar/cultura-digital)
    * [Otros programas](https://hacemosescuela.cba.gov.ar/programas)
  * [Comunidad](https://hacemosescuela.cba.gov.ar/comunidad-de-practicas)
  * [Conversatorios](https://hacemosescuela.cba.gov.ar/conversatorios)
  * Modalidades
    * [Jóvenes y Adultos](https://hacemosescuela.cba.gov.ar/jovenes-y-adultos)
    * [Educación Especial](https://hacemosescuela.cba.gov.ar/educacion-especial)
    * [Domiciliaria y Hospitalaria](https://hacemosescuela.cba.gov.ar/educacion-domiciliaria-y-hospitalaria)

![](https://hacemosescuela.cba.gov.ar/wp-content/uploads/2022/11/Banner-
materiales-educativos-1024x267.jpg)

##

## Materiales educativos

En este espacio encontrarán materiales ordenados por nivel educativo.<br>

**Node ID:** fb4c3f5c-a56b-494b-8f73-a9e2b05ede5c<br>**Similarity:** 0.7639715754382289<br>**Text:** Ofrece un abordaje acerca de un...

[](https://hacemosescuela.cba.gov.ar/te-invito-a-mi-jardin-conocer-y-
describir-nuestro-espacio-de-trabajo/)

[Te invito a mi jardín. Conocer y describir nuestro espacio de
trabajo](https://hacemosescuela.cba.gov.ar/te-invito-a-mi-jardin-conocer-y-
describir-nuestro-espacio-de-trabajo/)

“Te invito a mi jardín.<br>

**Node ID:** af14ee82-9691-4209-94a0-beca7774eba7<br>**Similarity:** 0.7621458538579383<br>**Text:** Patria, revolución y
guerra](https://hacemosescuela.cba.gov.ar/efemerides-25-de-mayo-patria-
revolucion-y-guerra/)

Efemérides: 25 de Mayo.<br>

**Node ID:** 05612c0d-7bd1-406e-9fea-ca818c3f0355<br>**Similarity:** 0.7618394834503764<br>**Text:** Paseos asombrosos, fotomontajes
lúdicos](https://hacemosescuela.cba.gov.ar/fotografia-en-el-aula-paseos-
asombrosos-fotomontajes-ludicos/)

“Fotografía en el aula. Paseos asombrosos, fotomontajes lúdicos” es una
secuencia didáctica de Hacemos Escuela destinada...

[](https://hacemosescuela.cba.gov.ar/de-la-colmena-a-tu-mesa/)

[De la colmena a tu mesa](https://hacemosescuela.cba.gov.ar/de-la-colmena-a-
tu-mesa/)

"De la colmena a tu mesa" es una secuencia didáctica de Tu Escuela en Casa
destinada a 1.º grado de nivel Primario....

[](https://hacemosescuela.cba.gov.ar/programamos-con-alternativa-condicional/)

[Programamos con alternativa
condicional](https://hacemosescuela.cba.gov.ar/programamos-con-alternativa-
condicional/)

“Programamos con alternativa condicional” es una clase de Hacemos Escuela
destinada al Primer Ciclo del nivel Primario. Propone...

[ __ Ver más materiales del nivel Primario
](http://hacemosescuela.cba.gov.ar/buscador/?nivel=161&key=)

## Educación Secundaria

[](https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-las-y-
los-trabajadores-derechos-reivindicaciones-y-luchas/)

[Efemérides: 1.º de Mayo El trabajo, las y los trabajadores: derechos,
reivindicaciones y
luchas](https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-
las-y-los-trabajadores-derechos-reivindicaciones-y-luchas/)

Efemérides: 1.º de Mayo.<br>

## Plug it into Query Engine


In [57]:
from llama_index.llms.gemini import Gemini

llm = Gemini()

In [58]:
from llama_index.core.query_engine import RetrieverQueryEngine

In [59]:
qa_prompt_str  = (
    "Contexto:\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n\n"
    "Usted es un ayudante y debe responder a la pregunta del usuario usando el contexto provisto.\n"
    "Responda  de forma amable, detallada y extensa.\n"
    "Además responda con enlaces a los recursos en caso de que sea posible.\n"
    """Si no encuentra la respuesta en el contexto responda: "No pude encontrar información en el contexto provisto".\n\n"""
    "Pregunta: {query_str}\n"
)

In [60]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

# Text QA Prompt
chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            "No responda si el contexto no contiene información suficiente."
        ),
    ),
    ChatMessage(role=MessageRole.USER, content=qa_prompt_str ),
]
text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

In [61]:
query_engine = RetrieverQueryEngine.from_args(retriever,
                                              llm=llm,
                                              text_qa_template=text_qa_template, 
                                              verbose=True, 
                                              callback_manager=callback_manager)
#base_query_engine = RetrieverQueryEngine.from_args(base_retriever)

In [62]:
query_str = (
    "qué material hay para la educación inicial?"
)

In [63]:
response = query_engine.query(query_str)

DEBUG:llama_index.core.indices.utils:> Top 6 nodes:
> [Node 14938e36-0656-484e-9cae-78e5eb78cbfb] [Similarity score:             0.792263] [ ![Logo](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/logo_202403...
> [Node 99094bec-32ae-4cc1-8e6c-2f306e917648] [Similarity score:             0.78047] Fueron
diseñados para diferentes escenarios escolares: presencial, no presencial o
combinado. Tie...
> [Node 7bce84fc-dbd0-4924-aa0b-55d6cb726e50] [Similarity score:             0.765475] es una secuencia didáctica de Tu
Escuela en Casa destinada a las salas de 4...

[ __ Ver más mate...
> [Node 79088fb8-d990-40b4-8554-980ac8080749] [Similarity score:             0.739715] [](https://hacemosescuela.cba.gov.ar/en-la-red-pero-sin-enredarme-parte-i/)

[En la red, pero sin...
> [Node fb4c3f5c-a56b-494b-8f73-a9e2b05ede5c] [Similarity score:             0.731224] Ofrece un abordaje acerca de un...

[](https://hacemosescuela.cba.gov.ar/te-invito-a-mi-jardin-co...
> [Node 1

In [64]:
print(str(response))

**Secuencias didácticas:**

* [25 de mayo: aquel primer festejo](https://hacemosescuela.cba.gov.ar/25-de-mayo-aquel-primer-festejo-4/)
* [Miradas pixeladas](https://hacemosescuela.cba.gov.ar/miradas-pixeladas/)
* [Te invito a mi jardín. Conocer y describir nuestro espacio de trabajo](https://hacemosescuela.cba.gov.ar/te-invito-a-mi-jardin-conocer-y-describir-nuestro-espacio-de-trabajo/)
* [Un recorrido por el jardín (Parte III)](https://hacemosescuela.cba.gov.ar/un-recorrido-por-el-jardin-parte-iii/)
* [Un recorrido por el jardín (Parte II)](https://hacemosescuela.cba.gov.ar/un-recorrido-por-el-jardin-parte-ii/)
* [Un recorrido por el jardín (Parte I)](https://hacemosescuela.cba.gov.ar/un-recorrido-por-el-jardin-parte-i/)
* [En la red, pero sin enredarme (Parte II)](https://hacemosescuela.cba.gov.ar/en-la-red-pero-sin-enredarme-parte-ii/)
* [En la red, pero sin enredarme (Parte I)](https://hacemosescuela.cba.gov.ar/en-la-red-pero-sin-enredarme-parte-i/)
* [Había una vez… ¡te lo cuento 

## Debugging

In [65]:
llama_debug.get_event_pairs()

[[CBEvent(event_type=<CBEventType.EMBEDDING: 'embedding'>, payload={<EventPayload.SERIALIZED: 'serialized'>: {'model_name': 'models/embedding-001', 'embed_batch_size': 10, 'num_workers': None, 'title': 'this is a document', 'task_type': 'retrieval_document', 'class_name': 'GeminiEmbedding'}}, time='05/08/2024, 12:24:55.909347', id_='b105a5d3-bb3b-4453-acbd-12c9a1905f98'),
  CBEvent(event_type=<CBEventType.EMBEDDING: 'embedding'>, payload={<EventPayload.CHUNKS: 'chunks'>: ['[ ![Logo](https://hacemosescuela.cba.gov.ar/wp-\ncontent/themes/hacemosescuela_2-8/img/logo_202403.png?v=20240411-1)\n](https://hacemosescuela.cba.gov.ar) __\n\n  * [Inicio](https://hacemosescuela.cba.gov.ar)\n  * [Acerca de](https://hacemosescuela.cba.gov.ar/acerca-de)\n  * Materiales educativos\n    * [Secuencias didácticas](https://hacemosescuela.cba.gov.ar/materiales-educativos)\n    * [Objetos Hipermediales Interactivos](https://hacemosescuela.cba.gov.ar/ohi)\n    * [Colecciones didácticas](https://hacemosescuel

In [66]:
llama_debug.get_llm_inputs_outputs()

[[CBEvent(event_type=<CBEventType.LLM: 'llm'>, payload={<EventPayload.MESSAGES: 'messages'>: [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='No responda si el contexto no contiene información suficiente.', additional_kwargs={}), ChatMessage(role=<MessageRole.USER: 'user'>, content='Contexto:\n---------------------\nes una secuencia didáctica de Tu\nEscuela en Casa destinada a las salas de 4...\n\n[ __ Ver más materiales del nivel Inicial\n](http://hacemosescuela.cba.gov.ar/buscador/?nivel=158&key=)\n\n## Educación Primaria\n\n[](https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-las-y-\nlos-trabajadores-derechos-reivindicaciones-y-luchas/)\n\n[Efemérides: 1.º de Mayo El trabajo, las y los trabajadores: derechos,\nreivindicaciones y\nluchas](https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-\nlas-y-los-trabajadores-derechos-reivindicaciones-y-luchas/)\n\nEfemérides: 1.º de Mayo. El trabajo, las y los trabajadores: derechos,\nreivindicaciones

In [67]:
event_pairs = llama_debug.get_llm_inputs_outputs()

print(event_pairs[0][0])
print(event_pairs[0][1].payload.keys())
print(event_pairs[0][1].payload["response"])

CBEvent(event_type=<CBEventType.LLM: 'llm'>, payload={<EventPayload.MESSAGES: 'messages'>: [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='No responda si el contexto no contiene información suficiente.', additional_kwargs={}), ChatMessage(role=<MessageRole.USER: 'user'>, content='Contexto:\n---------------------\nes una secuencia didáctica de Tu\nEscuela en Casa destinada a las salas de 4...\n\n[ __ Ver más materiales del nivel Inicial\n](http://hacemosescuela.cba.gov.ar/buscador/?nivel=158&key=)\n\n## Educación Primaria\n\n[](https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-las-y-\nlos-trabajadores-derechos-reivindicaciones-y-luchas/)\n\n[Efemérides: 1.º de Mayo El trabajo, las y los trabajadores: derechos,\nreivindicaciones y\nluchas](https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-\nlas-y-los-trabajadores-derechos-reivindicaciones-y-luchas/)\n\nEfemérides: 1.º de Mayo. El trabajo, las y los trabajadores: derechos,\nreivindicaciones y

In [68]:
print(event_pairs[0][0])
print(event_pairs[0][0].payload.keys())


CBEvent(event_type=<CBEventType.LLM: 'llm'>, payload={<EventPayload.MESSAGES: 'messages'>: [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='No responda si el contexto no contiene información suficiente.', additional_kwargs={}), ChatMessage(role=<MessageRole.USER: 'user'>, content='Contexto:\n---------------------\nes una secuencia didáctica de Tu\nEscuela en Casa destinada a las salas de 4...\n\n[ __ Ver más materiales del nivel Inicial\n](http://hacemosescuela.cba.gov.ar/buscador/?nivel=158&key=)\n\n## Educación Primaria\n\n[](https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-las-y-\nlos-trabajadores-derechos-reivindicaciones-y-luchas/)\n\n[Efemérides: 1.º de Mayo El trabajo, las y los trabajadores: derechos,\nreivindicaciones y\nluchas](https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-\nlas-y-los-trabajadores-derechos-reivindicaciones-y-luchas/)\n\nEfemérides: 1.º de Mayo. El trabajo, las y los trabajadores: derechos,\nreivindicaciones y

In [71]:
print(event_pairs[0][0].payload["messages"][1])

user: Contexto:
---------------------
es una secuencia didáctica de Tu
Escuela en Casa destinada a las salas de 4...

[ __ Ver más materiales del nivel Inicial
](http://hacemosescuela.cba.gov.ar/buscador/?nivel=158&key=)

## Educación Primaria

[](https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-las-y-
los-trabajadores-derechos-reivindicaciones-y-luchas/)

[Efemérides: 1.º de Mayo El trabajo, las y los trabajadores: derechos,
reivindicaciones y
luchas](https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-
las-y-los-trabajadores-derechos-reivindicaciones-y-luchas/)

Efemérides: 1.º de Mayo. El trabajo, las y los trabajadores: derechos,
reivindicaciones y luchas es una compilación de registros...

[](https://hacemosescuela.cba.gov.ar/efemerides-25-de-mayo-aquel-primer-
festejo/)

[Efemérides: 25 de mayo.

[ ![Logo](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/logo_202403.png?v=20240411-1)
](https://hacemosescuela.cba.gov.ar

In [70]:
# Clear the currently cached events
llama_debug.flush_event_logs()